In [46]:
import os
import time

from importlib import reload

import pickle as pk
import pandas as pd

import numpy as np
from scipy.stats import norm as norm_dist

from sklearn.cluster import KMeans
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import KFold

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed

#sys.path.extend([os.getcwd()])

from calibre.model import gaussian_process as gp
from calibre.model import tailfree_process as tail_free
from calibre.model import gp_regression_monotone as gpr_mono
from calibre.model import adaptive_ensemble

from calibre.inference import mcmc

from calibre.calibration import score

import calibre.util.misc as misc_util
import calibre.util.metric as metric_util
import calibre.util.visual as visual_util
import calibre.util.matrix as matrix_util
import calibre.util.ensemble as ensemble_util
import calibre.util.calibration as calib_util

import calibre.util.experiment_pred as pred_util

from calibre.util.inference import make_value_setter

import matplotlib.pyplot as plt
import seaborn as sns
import shapefile as shp
import pathlib

In [54]:
_MODEL_DICTIONARY = {"root": ["AV", "GS", "CM"]}

In [55]:
y_obs = pd.read_csv("CA_2010_2016/CA_clean_training_2010_2016.csv")

X_train = np.asarray(y_obs[["lon", "lat", "time"]].values.tolist()).astype(np.float32)

base_train_feat = dict()

for model_name in tail_free.get_leaf_model_names(_MODEL_DICTIONARY):
    base_train_feat[model_name] = X_train
   

""" 1. prepare prediction data dictionary """
base_valid_feat = dict()


for model_name in tail_free.get_leaf_model_names(_MODEL_DICTIONARY):
    data_pd = pd.read_csv("CA_2010_2016/{}_2010_2016_align.csv".format('CA_'+ model_name))
    base_valid_feat[model_name] = np.asarray(data_pd[["lon", "lat","time"]].values.tolist()).astype(np.float32)
 
X_valid = base_valid_feat[model_name]

""" 3. standardize data """
# standardize
X_centr = np.mean(X_valid, axis=0)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

X_scale_dist = np.max(X_scale[0:1])
X_scale[0] = X_scale_dist
X_scale[1] = X_scale_dist

#X_valid = (X_valid - X_centr) / X_scale
# X_train = (X_train - X_centr) / X_scale

In [56]:
#To load from pickle file

def load_pk(file_name):
  data = []

  for i in range(84):

    with open('CA_2010_2016/hmc'+str(i)+'/'+file_name, 'rb') as f:
        try:
            while True:
                data.append(pk.load(f))
        except EOFError:
            pass
  return data 
  
def get_overlapping_average(a):
    df = pd.DataFrame(a)
    out = df.groupby([0, 1, 2]).mean().reset_index()
    return out.values

ensemble_sample_val_mean = get_overlapping_average(np.concatenate(load_pk('ensemble_sample_val_mean.pkl'), axis=0))

#print(ensemble_sample_val_mean.shape) 
#(723716, 4)
#the 3rd column should be time
#print(ensemble_sample_val_mean)


ensemble_mean_val_mean = get_overlapping_average(np.concatenate(load_pk('ensemble_mean_val_mean.pkl'), axis=0))
mean_resid = get_overlapping_average(np.concatenate(load_pk('mean_resid.pkl'), axis=0))

ensemble_sample_val_var = get_overlapping_average(np.concatenate(load_pk('ensemble_sample_val_var.pkl'), axis=0))
ensemble_mean_val_var = get_overlapping_average(np.concatenate(load_pk('ensemble_mean_val_var.pkl'), axis=0))
uncn_resid = get_overlapping_average(np.concatenate(load_pk('uncn_resid.pkl'), axis=0))
uncn_noise = get_overlapping_average(np.concatenate(load_pk('uncn_noise.pkl'), axis=0))

ensemble_weights_val = get_overlapping_average(np.concatenate(load_pk('ensemble_weights_val.pkl'), axis=0))


def filter_year(df, year):
    return df[np.where(df[:, 2]==year)]

#get unique years 
years = set(ensemble_sample_val_mean[:, 2])



In [57]:
sf = shp.Reader('tl_2017_us_state/tl_2017_us_state.shp')
sf_df = pd.DataFrame(sf.records())
state_sf = np.take(sf.shapeRecords(), sf_df[sf_df[5].isin(['CA', 'NV', 'AZ'])].index)

In [58]:
def my_posterior_heatmap_2d(plot_data, X,
                         X_monitor=None,
                         cmap='inferno_r',
                         norm=None, norm_method="percentile",
                         save_addr=''):
    
    if save_addr:
        pathlib.Path(save_addr).parent.mkdir(parents=True, exist_ok=True)
        plt.ioff()

    if not norm:
        norm = make_color_norm(plot_data, method=norm_method)

    # 2d color plot using scatter
    # made changes here on the fig size
    plt.figure(figsize=(10, 8))
    plt.scatter(x=X[:, 0], y=X[:, 1],
                s=3,
                c=plot_data, cmap=cmap, norm=norm)
    cbar = plt.colorbar()
    

    # plot monitors
    if isinstance(X_monitor, np.ndarray):
        plt.scatter(x=X_monitor[:, 0], y=X_monitor[:, 1],
                    s=10, c='black')

    # adjust plot window
    plt.xlim((np.min(X[:, 0]), np.max(X[:, 0])))
    plt.ylim((np.min(X[:, 1]), np.max(X[:, 1])))
    
    #overlap the state borders
    for shape in state_sf:
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
        plt.scatter(x,y,color='black',s=0.01)
    
    
    if save_addr:
        plt.savefig(save_addr, bbox_inches='tight')
        plt.close()
        plt.ion()
    else:
        plt.show()

    return norm


In [59]:
color_norm_pred = visual_util.make_color_norm(
        ensemble_sample_val_mean[:,3],  # exclude "resid" vales from pal
        method="percentile")
# prepare color norms for plt.scatter
color_norm_unc = visual_util.make_color_norm(
        ensemble_sample_val_var[:,3],  # use "overall" and "mean" for pal
        method="percentile")
# prepare color norms for plt.scatter
color_norm_weights = visual_util.make_color_norm(
        ensemble_weights_val[:, 3:],  
        method="percentile")


In [62]:
for year in years:
    
    _SAVE_ADDR_PREFIX_curr = 'CA_2010_2016/new_'+str(int(year))
    X_train_curr = filter_year(X_train, year)
    
    #getting the re-ordered lat and lon
    X_valid_reordered = filter_year(ensemble_sample_val_mean, year)[:, :3]
#     X_valid_reordered = (X_valid_reordered - X_centr) / X_scale
    X_valid_reordered_locations = X_valid_reordered[:, :2]
    #(723716, 2)

    
    post_mean_dict = {
        "overall": filter_year(ensemble_sample_val_mean, year)[:, 3],
        "mean": filter_year(ensemble_mean_val_mean, year)[:, 3],
        "resid": filter_year(mean_resid, year)[:, 3]
    }

    post_uncn_dict = {
        "overall": filter_year(ensemble_sample_val_var, year)[:, 3],
        "mean": filter_year(ensemble_mean_val_var, year)[:, 3],
        "resid": filter_year(uncn_resid, year)[:, 3],
        "noise": filter_year(uncn_noise, year)[:, 3]
    }


    weights_dict = {}
    model_names = _MODEL_DICTIONARY['root']
    
    for i in range(len(model_names)): 
      weights_dict[model_names[i]] = filter_year(ensemble_weights_val, year)[:, i+3]

    #print(color_norm_unc)

    color_norm_ratio = visual_util.make_color_norm(
        post_uncn_dict["noise"] / post_uncn_dict["overall"],
        method="percentile")
    

    """ 3.1. posterior predictive uncertainty """
    for unc_name, unc_value in post_uncn_dict.items():
        save_name = os.path.join(_SAVE_ADDR_PREFIX_curr,
                                 'ensemble_posterior_uncn_{}.png'.format(
                                     unc_name))

        color_norm = my_posterior_heatmap_2d(unc_value,
                                                      X=X_valid_reordered_locations, X_monitor=X_train_curr,
                                                      cmap='inferno_r',
                                                      norm=color_norm_unc,
                                                      norm_method="percentile",
                                                      save_addr=save_name)

    """ 3.2. posterior predictive mean """
    for mean_name, mean_value in post_mean_dict.items():
        save_name = os.path.join(_SAVE_ADDR_PREFIX_curr,
                                 'ensemble_posterior_mean_{}.png'.format(
                                    mean_name))
        color_norm =my_posterior_heatmap_2d(mean_value,
                                                      X=X_valid_reordered_locations, X_monitor=X_train_curr,
                                                      cmap='RdYlGn_r',
                                                      norm=color_norm_pred,
                                                      norm_method="percentile",
                                                      save_addr=save_name)



    """ 3.3. model weights """

    for model_name, model_weight in weights_dict.items():
        save_name = os.path.join(_SAVE_ADDR_PREFIX_curr,
                                 'ensemble_weights_val_{}.png'.format(
                                      model_name))

        color_norm = my_posterior_heatmap_2d(model_weight,
                                                      X=X_valid_reordered_locations, X_monitor=X_train_curr,
                                                      cmap='viridis',
                                                      norm=color_norm_weights,
                                                      norm_method="percentile",
                                                      save_addr=save_name)



In [121]:
overall_bne_pred = pd.DataFrame(ensemble_sample_val_mean)

In [122]:
overall_bne_pred.shape

(5060384, 4)

In [123]:
overall_bne_pred = overall_bne_pred.rename(columns={0:'lon', 1:'lat', 2:'time', 3:'bne_pred'})

In [124]:
overall_bne_pred.to_csv('CA_2010_2016/CA_2010_2016_ensemble_pred.csv')